<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D2_MongoDB_Remote_Complex_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Purpose
Here we demonstrate two things 


1.   How to connect to a remote MongoDB server on Atlas servers
2.   How to load bulk data and execute complex queries

Some output has been suppressed by commenting out the printcommands in certain cells. This was necessary to reduce the size of the file for upload to github



Additional Material https://github.com/giffy/MongoDB_PyMongo_Tutorial

In [1]:
from datetime import datetime
import pytz
print('Tested on ',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

Tested on  2023-05-05 10:16:36.154590+05:30
Python 3.10.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.5 LTS
Release:	20.04
Codename:	focal


#Install MongoDB

In [2]:
#!pip3 install pymongo > /dev/null
#!/usr/bin/python3 -m pip -qq install "pymongo[srv]"             # Required to access remote MongoDB
#from pymongo import MongoClient

#!apt install mongodb > /dev/null
#!apt install mongodb 

#!python -m pip install pymongo==3.6
!python -m pip install pymongo

from pymongo.mongo_client import MongoClient
#from pymongo.server_api import ServerApi

import pandas as pd
import dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 23.5 MB/s eta 0:00:00


In [4]:
# Install database tools to get mongo import

# https://www.mongodb.com/try/download/database-tools
# https://www.mongodb.com/docs/database-tools/installation/installation-linux/

#!wget https://fastdl.mongodb.org/tools/db/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.tgz
!wget -q -O tools.deb https://fastdl.mongodb.org/tools/db/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.deb

#!tar -xvf /content/mongodb-database-tools-ubuntu2004-x86_64-100.7.0.tgz
#!sudo apt install ./mongodb-database-tools-*-100.7.0.deb
!apt install ./tools.deb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'mongodb-database-tools' instead of './tools.deb'
The following NEW packages will be installed:
  mongodb-database-tools
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 0 B/50.6 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 /content/tools.deb mongodb-database-tools amd64 100.7.0 [50.6 MB]
Selecting previously unselected package mongodb-database-tools.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack /content/tools.deb ...
Unpacking mongodb-database-tools (100.7.0) ...
Setting up mongodb-database-tools (100.7.0) ...


In [ ]:
#Local Server Need Not be started for Remote Operations
#!service mongodb start

## Test MongoClient with Remote Server

In [ ]:
# The Password is visible in this cell
# If you wish to hide this then use the next cell
#
#rhostName = 'something-mongodb.services.clever-cloud.com'   # place your own credentials here
#ruserName = 'hidden'      # place your own credentials here
#rpassWord = 'hidden'      # place your own credentials here
#rdbName =  'something'         # place your own credentials here
#rMongoURI = 'hidden'         # place your own credentials here

### Collect Credentials from G-Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/'My Drive'/Praxis/WebCredentials


Mounted at /content/drive
ASTRA_DataStaxCredentials_Jun22.py  QuandlApiRegistrations_praxis.txt
AtlasCredentials_Jun22.py	    QuandlApiRegistrations.txt
CCPraxisMongoDB16Oct22.py	    secure-connect-praxisdb.zip
cleverCloudMongo51Sep22.py	    zzclevercloudMongoDB_jun22.py
cleverCloud.py			    zzclevercloudMongoDB.py
db4freeCredentials.py		    zzparashar21URI.py
ngrokToken.py			    zzsqlCredentials_020221.py-deprecated
Parashar21_Atlas_18Sep22.py


In [7]:
# copy credentials file

#!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB_jun22.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/cleverCloudMongo51Sep22.py credentials.py
#!cp /content/drive/'My Drive'/Praxis/WebCredentials/CCPraxisMongoDB16Oct22.py credentials.py

!cp /content/drive/'My Drive'/Praxis/WebCredentials/AtlasCredentials_Jun22.py credentials.py



In [ ]:
#!cat credentials.py

In [8]:
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

#from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI
from credentials import RegCalURI
rMongoURI = RegCalURI

#for the sake of privacy
#the following credentials need to be stored in a text file called sqlCredentials.py
#in the format given below
#in the Colab VM

#otherwise, the values can be directly placed here

#rhostName = 'rs0/n1-c2-mongodb-clevercloud-customers.services.clever-cloud.com:27017,n2-c2-mongodb-clevercloud-customers.services.clever-cloud.com:27017'   # place your own credentials here
#ruserName = 'ubdzzfg3r4qhpu0kg1vd'      # place your own credentials here
#rpassWord = 'YzIIYhyTJjOJNCkkntPp'      # place your own credentials here
#rdbName =  'bcgygtzxy3oqpm3'         # place your own credentials here


#print(rhostName,ruserName,rdbName)
#print(rMongoURI)

In [9]:
mongoURI = rMongoURI                                   # for remote server
#mongoURI = "mongodb://localhost:27017/"                 # for local server
#client = MongoClient(mongoURI, server_api=ServerApi('1'))
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['Parashar21', 'PraxisDB', 'admin', 'local']

In [10]:
#db = client.PraxisDB
#db = client.test
db = client.PraxisDB
db.list_collection_names()

['restaurants']

In [11]:
table = db.restaurants
table.count_documents({})

3772

In [12]:
# Delete collection
db.drop_collection('restaurants')

{'nIndexesWas': 1,
 'ns': 'PraxisDB.restaurants',
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1683262369, 15),
  'signature': {'hash': b'\xb3\xcdo6s9m\xad\n\xfb\xc7J_\xe8\xfaH\xe0\x89]\xe9',
   'keyId': 7186598201770639362}},
 'operationTime': Timestamp(1683262369, 15)}

#MongoDB Exercise <br>
Exercise : https://docs.google.com/document/d/1q0PyJL6EgMG1V5G8mmPre8Ypq9N9dX-PRLixAHiphd8/edit?usp=sharing <br>

Data : restaurant.json https://drive.google.com/file/d/14sYJM2KwIpDRsAAk6YkapMTwLnLuBcb_/view?usp=sharing <br>

For more detailed examples, see https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/1_1_First_steps_with_MongoDB_PyMongo.ipynb

##Get Data

In [13]:
#!gdown https://drive.google.com/uc?id=14sYJM2KwIpDRsAAk6YkapMTwLnLuBcb_
#!gdown 14sYJM2KwIpDRsAAk6YkapMTwLnLuBcb_
!wget -q https://raw.githubusercontent.com/Praxis-QR/BDSN/4130bfe8fb17529bbeae32351dab982b4a8b53ea/data/restaurants.json

##Load Data

In [14]:
# the restaurants.json file is available in my GDrive
# Local Server
#!mongoimport --db PraxisDB --collection restaurants --drop --file restaurants.json
# Remote Server
# mongoimport --host "hostname" --port "port" --db "databasename" --collection "collectionName" --file "filePath"
# the python variables need to be placed in {}
#!mongoimport --host "{rhostName}" --port "27017" --db "{rdbName}" --collection "restaurants"  --drop --file "restaurants.json" -u "{ruserName}" -p "{rpassWord}" --- THIS DOES NOT WORK
#!mongoimport --uri "{rMongoURI}"  --collection "restaurants"  --drop --file "restaurants.json"



# https://stackoverflow.com/questions/63224330/where-do-i-find-my-dbname-for-mongodb-connection-string
#mongoURI2 = mongoURI
#print(mongoURI)
mongoURI2 = mongoURI.replace("/?retryWrites=true&w=majority","/PraxisDB?")
#print(mongoURI2)
!mongoimport --uri "{mongoURI2}"  --collection "restaurants"  --drop --file "restaurants.json" 

2023-05-05T04:53:05.758+0000	connected to: mongodb+srv://[**REDACTED**]@cluster0.yarln.mongodb.net/PraxisDB?
2023-05-05T04:53:05.983+0000	dropping: PraxisDB.restaurants
2023-05-05T04:53:08.759+0000	[######..................] PraxisDB.restaurants	538KB/1.94MB (27.1%)
2023-05-05T04:53:11.758+0000	[############............] PraxisDB.restaurants	1.04MB/1.94MB (53.6%)
2023-05-05T04:53:14.758+0000	[###################.....] PraxisDB.restaurants	1.55MB/1.94MB (80.0%)
2023-05-05T04:53:17.758+0000	[########################] PraxisDB.restaurants	1.94MB/1.94MB (100.0%)
2023-05-05T04:53:20.087+0000	[########################] PraxisDB.restaurants	1.94MB/1.94MB (100.0%)
2023-05-05T04:53:20.087+0000	3772 document(s) imported successfully. 0 document(s) failed to import.


##Data Access & Manipulation

In [15]:
#mongoURI = "mongodb://localhost:27017/"    # local
#mongoURI = rMongoURI                        # remote
client = MongoClient(mongoURI)
client.list_database_names() 

['Parashar21', 'PraxisDB', 'admin', 'local']

In [16]:
#db = client.PraxisDB
#db = client.btncgfi3a5bxyea
db = client.PraxisDB
db.list_collection_names()

['restaurants']

In [17]:
table = db.restaurants
table.count_documents({})

3772

# 32 Diverse Queries

1. display all the documents in the collection restaurants


In [18]:
c1 = db.restaurants.find().limit(3)
for crnt in c1:
    print(crnt)

{'_id': ObjectId('64548bb29d4605c517866b61'), 'address': {'building': '469', 'coord': [-73.961704, 40.662942], 'street': 'Flatbush Avenue', 'zipcode': '11225'}, 'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'grades': [{'date': datetime.datetime(2014, 12, 30, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2014, 7, 1, 0, 0), 'grade': 'B', 'score': 23}, {'date': datetime.datetime(2013, 4, 30, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}], 'name': "Wendy'S", 'restaurant_id': '30112340'}
{'_id': ObjectId('64548bb29d4605c517866b62'), 'address': {'building': '1007', 'coord': [-73.856077, 40.848447], 'street': 'Morris Park Ave', 'zipcode': '10462'}, 'borough': 'Bronx', 'cuisine': 'Bakery', 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0), 'grade': 'A', 'score': 2}, {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6}, {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10

2. display the fields restaurant_id, name, borough and cuisine for all the documents in the collection restaurant. 


In [19]:
c1 = db.restaurants.find({},{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1}).limit(3)
for crnt in c1:
    print(crnt)

{'_id': ObjectId('64548bb29d4605c517866b61'), 'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'name': "Wendy'S", 'restaurant_id': '30112340'}
{'_id': ObjectId('64548bb29d4605c517866b62'), 'borough': 'Bronx', 'cuisine': 'Bakery', 'name': 'Morris Park Bake Shop', 'restaurant_id': '30075445'}
{'_id': ObjectId('64548bb29d4605c517866b63'), 'borough': 'Brooklyn', 'cuisine': 'American ', 'name': 'Riviera Caterer', 'restaurant_id': '40356018'}


3. display the fields restaurant_id, name, borough and cuisine, but exclude the field _id for all the documents in the collection restaurant. 


In [ ]:
c1 = db.restaurants.find({},{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1,"_id":0}).limit(3)
for crnt in c1:
    print(crnt)

{'borough': 'Brooklyn', 'cuisine': 'Hamburgers', 'name': "Wendy'S", 'restaurant_id': '30112340'}
{'borough': 'Bronx', 'cuisine': 'Bakery', 'name': 'Morris Park Bake Shop', 'restaurant_id': '30075445'}
{'borough': 'Brooklyn', 'cuisine': 'American ', 'name': 'Riviera Caterer', 'restaurant_id': '40356018'}


4. display the fields restaurant_id, name, borough and zipcode, but exclude the field _id for all the documents in the collection restaurant. 

In [ ]:
table = db.restaurants
c1 = table.find({},{"restaurant_id" : 1,"name":1,"borough":1,"address.zipcode" :1,"_id":0}).limit(3)
for crnt in c1:
    print(crnt)

{'address': {'zipcode': '11225'}, 'borough': 'Brooklyn', 'name': "Wendy'S", 'restaurant_id': '30112340'}
{'address': {'zipcode': '10462'}, 'borough': 'Bronx', 'name': 'Morris Park Bake Shop', 'restaurant_id': '30075445'}
{'address': {'zipcode': '11224'}, 'borough': 'Brooklyn', 'name': 'Riviera Caterer', 'restaurant_id': '40356018'}


In [ ]:
#df = pd.DataFrame(list(table.find({},{"restaurant_id" : 1,"name":1,"borough":1,"address.zipcode" :1,"_id":0}).limit(10)))
#df
pd.DataFrame(list(table.find({},{"restaurant_id" : 1,"name":1,"borough":1,"address.zipcode" :1,"_id":0}).limit(10)))

,address,borough,name,restaurant_id
0,{'zipcode': '11225'},Brooklyn,Wendy'S,30112340
1,{'zipcode': '10462'},Bronx,Morris Park Bake Shop,30075445
2,{'zipcode': '11224'},Brooklyn,Riviera Caterer,40356018
3,{'zipcode': '10019'},Manhattan,Dj Reynolds Pub And Restaurant,30191841
4,{'zipcode': '11374'},Queens,Tov Kosher Kitchen,40356068
5,{'zipcode': '10314'},Staten Island,Kosher Island,40356442
6,{'zipcode': '11234'},Brooklyn,Wilken'S Fine Food,40356483
7,{'zipcode': '11369'},Queens,Brunos On The Boulevard,40356151
8,{'zipcode': '11219'},Brooklyn,Regina Caterers,40356649
9,{'zipcode': '11226'},Brooklyn,Taste The Tropics Ice Cream,40356731


5. display all the restaurant which is in the borough Bronx

In [ ]:
c1 = table.find({"borough": "Bronx"}).limit(3)
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875d0d'), 'address': {'building': '1007', 'coord': [-73.856077, 40.848447], 'street': 'Morris Park Ave', 'zipcode': '10462'}, 'borough': 'Bronx', 'cuisine': 'Bakery', 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0), 'grade': 'A', 'score': 2}, {'date': datetime.datetime(2013, 9, 11, 0, 0), 'grade': 'A', 'score': 6}, {'date': datetime.datetime(2013, 1, 24, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2011, 11, 23, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2011, 3, 10, 0, 0), 'grade': 'B', 'score': 14}], 'name': 'Morris Park Bake Shop', 'restaurant_id': '30075445'}
{'_id': ObjectId('634b6160f53e839bcf875d16'), 'address': {'building': '2300', 'coord': [-73.8786113, 40.8502883], 'street': 'Southern Boulevard', 'zipcode': '10460'}, 'borough': 'Bronx', 'cuisine': 'American ', 'grades': [{'date': datetime.datetime(2014, 5, 28, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2013, 6, 19, 0, 0), 'grade'

In [ ]:
df = pd.DataFrame(list(table.find({"borough": "Bronx"}).limit(3)))
df

,_id,address,borough,cuisine,grades,name,restaurant_id
0,634b6160f53e839bcf875d0d,"{'building': '1007', 'coord': [-73.856077, 40....",Bronx,Bakery,"[{'date': 2014-03-03 00:00:00, 'grade': 'A', '...",Morris Park Bake Shop,30075445
1,634b6160f53e839bcf875d16,"{'building': '2300', 'coord': [-73.8786113, 40...",Bronx,American,"[{'date': 2014-05-28 00:00:00, 'grade': 'A', '...",Wild Asia,40357217
2,634b6160f53e839bcf875d2a,"{'building': '1006', 'coord': [-73.84856870000...",Bronx,"Ice Cream, Gelato, Yogurt, Ices","[{'date': 2014-04-24 00:00:00, 'grade': 'A', '...",Carvel Ice Cream,40363093


7. display the next 5 restaurants after skipping first 5 which are in the borough Bronx. 


In [ ]:
pd.DataFrame(list(table.find({"borough": "Bronx"}).skip(5).limit(3)))

,_id,address,borough,cuisine,grades,name,restaurant_id
0,634b6160f53e839bcf875d49,"{'building': '658', 'coord': [-73.813639999999...",Bronx,American,"[{'date': 2014-06-21 00:00:00, 'grade': 'A', '...",Manhem Club,40364363
1,634b6160f53e839bcf875d61,"{'building': '2222', 'coord': [-73.84971759999...",Bronx,American,"[{'date': 2014-12-18 00:00:00, 'grade': 'A', '...",The New Starling Athletic Club Of The Bronx,40364956
2,634b6160f53e839bcf875d7a,"{'building': '72', 'coord': [-73.92506, 40.827...",Bronx,American,"[{'date': 2014-04-15 00:00:00, 'grade': 'A', '...",Yankee Tavern,40365499


8. find the restaurants who achieved a score more than 90. <hr>
https://stackoverflow.com/questions/68772027/mongodb-with-python-on-google-colab-syntax-error-on-elemmatch/68796638#68796638

In [ ]:
c1 = db.restaurants.find({"grades" : { "$elemMatch":{"score":{"$gt" : 90}}}})
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875e69'), 'address': {'building': '65', 'coord': [-73.9782725, 40.7624022], 'street': 'West   54 Street', 'zipcode': '10019'}, 'borough': 'Manhattan', 'cuisine': 'American ', 'grades': [{'date': datetime.datetime(2014, 8, 22, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2014, 3, 28, 0, 0), 'grade': 'C', 'score': 131}, {'date': datetime.datetime(2013, 9, 25, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2013, 4, 8, 0, 0), 'grade': 'B', 'score': 25}, {'date': datetime.datetime(2012, 10, 15, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2011, 10, 19, 0, 0), 'grade': 'A', 'score': 13}], 'name': "Murals On 54/Randolphs'S", 'restaurant_id': '40372466'}
{'_id': ObjectId('634b6160f53e839bcf875f0b'), 'address': {'building': '345', 'coord': [-73.9864626, 40.7266739], 'street': 'East 6 Street', 'zipcode': '10003'}, 'borough': 'Manhattan', 'cuisine': 'Indian', 'grades': [{'date': datetime.datetime(2014, 9, 15, 0, 

In [ ]:
c1 = db.restaurants.find({"grades" : { "$elemMatch":{"score":{"$gt" : 90}}}},{"name":1,"borough":1,"address.zipcode" :1,"_id":0})
for crnt in c1:
    print(crnt)

{'address': {'zipcode': '10019'}, 'borough': 'Manhattan', 'name': "Murals On 54/Randolphs'S"}
{'address': {'zipcode': '10003'}, 'borough': 'Manhattan', 'name': 'Gandhi'}
{'address': {'zipcode': '10016'}, 'borough': 'Manhattan', 'name': 'Bella Napoli'}


9. find the restaurants that achieved a score, more than 80 but less than 100. 


In [ ]:
c1 = db.restaurants.find({"grades" : { "$elemMatch":{"score":{"$gt" : 80 , "$lt" :100}}}})
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875f0b'), 'address': {'building': '345', 'coord': [-73.9864626, 40.7266739], 'street': 'East 6 Street', 'zipcode': '10003'}, 'borough': 'Manhattan', 'cuisine': 'Indian', 'grades': [{'date': datetime.datetime(2014, 9, 15, 0, 0), 'grade': 'A', 'score': 5}, {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2013, 5, 30, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 4, 24, 0, 0), 'grade': 'P', 'score': 2}, {'date': datetime.datetime(2012, 10, 1, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 4, 6, 0, 0), 'grade': 'C', 'score': 92}, {'date': datetime.datetime(2011, 11, 3, 0, 0), 'grade': 'C', 'score': 41}], 'name': 'Gandhi', 'restaurant_id': '40381295'}
{'_id': ObjectId('634b6160f53e839bcf876070'), 'address': {'building': '130', 'coord': [-73.984758, 40.7457939], 'street': 'Madison Avenue', 'zipcode': '10016'}, 'borough': 'Manhattan', 'cuisine': 'Pizza/Italian', 'g

10. find the restaurants which locates in latitude value less than -95.754168. 


In [ ]:
c1 = db.restaurants.find({"address.coord" : {"$lt" : -95.754168}})
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf8763ac'), 'address': {'building': '3707', 'coord': [-101.8945214, 33.5197474], 'street': '82 Street', 'zipcode': '11372'}, 'borough': 'Queens', 'cuisine': 'American ', 'grades': [{'date': datetime.datetime(2014, 6, 4, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 11, 7, 0, 0), 'grade': 'B', 'score': 19}, {'date': datetime.datetime(2013, 5, 17, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2012, 8, 29, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2012, 4, 3, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2011, 11, 16, 0, 0), 'grade': 'A', 'score': 7}], 'name': 'Burger King', 'restaurant_id': '40534067'}
{'_id': ObjectId('634b6160f53e839bcf87674a'), 'address': {'building': '15259', 'coord': [-119.6368672, 36.2504996], 'street': '10 Avenue', 'zipcode': '11357'}, 'borough': 'Queens', 'cuisine': 'Italian', 'grades': [{'date': datetime.datetime(2014, 9, 4, 0, 0), 'grade': 'A', 'score': 1

11. find the restaurants that does not prepare any cuisine of 'American' and their grade score more than 70 and latitude less than -65.754168.

In [ ]:
c1 = db.restaurants.find(
               {"$and":
                    [
                       {"cuisine" : {"$ne" :"American "}},
                       {"grades.score" : {"$gt" : 70}},
                       {"address.coord" : {"$lt" : -65.754168}}
                    ]
                },
                {"restaurant_id" : 1,"name":1,"borough":1,"address.zipcode" :1,"_id":0}
                    );
for crnt in c1:
    print(crnt)

{'address': {'zipcode': '10003'}, 'borough': 'Manhattan', 'name': 'Gandhi', 'restaurant_id': '40381295'}
{'address': {'zipcode': '10016'}, 'borough': 'Manhattan', 'name': 'Bella Napoli', 'restaurant_id': '40393488'}
{'address': {'zipcode': '10451'}, 'borough': 'Bronx', 'name': 'El Molino Rojo Restaurant', 'restaurant_id': '40393688'}
{'address': {'zipcode': '11211'}, 'borough': 'Brooklyn', 'name': 'Fortunato Bros Cafe & Bakery', 'restaurant_id': '40400561'}
{'address': {'zipcode': '10017'}, 'borough': 'Manhattan', 'name': 'Two Boots Grand Central', 'restaurant_id': '40725591'}


In [ ]:
whereCond = {"$and":
                    [
                       {"cuisine" : {"$ne" :"American "}},
                       {"grades.score" : {"$gt" : 70}},
                       {"address.coord" : {"$lt" : -65.754168}}
                    ]
                }
selFields = {
    "restaurant_id" : 1,
    "name":1,
    "borough":1,
    "address.zipcode" :1,
    "_id":0}

df = pd.DataFrame(db.restaurants.find(whereCond,selFields))
df

,address,borough,name,restaurant_id
0,{'zipcode': '10003'},Manhattan,Gandhi,40381295
1,{'zipcode': '10016'},Manhattan,Bella Napoli,40393488
2,{'zipcode': '10451'},Bronx,El Molino Rojo Restaurant,40393688
3,{'zipcode': '11211'},Brooklyn,Fortunato Bros Cafe & Bakery,40400561
4,{'zipcode': '10017'},Manhattan,Two Boots Grand Central,40725591


In [ ]:
c1 = db.restaurants.find(
               {"$and":
                    [
                       {"cuisine" : {"$ne" :"American "}},
                       {"grades.score" : {"$gt" : 70}},
                       {"address.coord" : {"$lt" : -65.754168}}
                    ]
                }
                    );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875f0b'), 'address': {'building': '345', 'coord': [-73.9864626, 40.7266739], 'street': 'East 6 Street', 'zipcode': '10003'}, 'borough': 'Manhattan', 'cuisine': 'Indian', 'grades': [{'date': datetime.datetime(2014, 9, 15, 0, 0), 'grade': 'A', 'score': 5}, {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2013, 5, 30, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 4, 24, 0, 0), 'grade': 'P', 'score': 2}, {'date': datetime.datetime(2012, 10, 1, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 4, 6, 0, 0), 'grade': 'C', 'score': 92}, {'date': datetime.datetime(2011, 11, 3, 0, 0), 'grade': 'C', 'score': 41}], 'name': 'Gandhi', 'restaurant_id': '40381295'}
{'_id': ObjectId('634b6160f53e839bcf876070'), 'address': {'building': '130', 'coord': [-73.984758, 40.7457939], 'street': 'Madison Avenue', 'zipcode': '10016'}, 'borough': 'Manhattan', 'cuisine': 'Pizza/Italian', 'g

12. find the restaurants which does not prepare any cuisine of 'American' and achieved a score more than 70 and not located in the longitude less than -65.754168.
Note : Do this query without using $and operator. 


In [ ]:
c1 = db.restaurants.find(
                     {"$query":
                            {
                             "cuisine" : {"$ne" : "American "},
                             "grades.score" :{"$gt": 70},
                             "address.coord" : {"$lt" : -65.754168}
                            }
                      });
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875f0b'), 'address': {'building': '345', 'coord': [-73.9864626, 40.7266739], 'street': 'East 6 Street', 'zipcode': '10003'}, 'borough': 'Manhattan', 'cuisine': 'Indian', 'grades': [{'date': datetime.datetime(2014, 9, 15, 0, 0), 'grade': 'A', 'score': 5}, {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2013, 5, 30, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2013, 4, 24, 0, 0), 'grade': 'P', 'score': 2}, {'date': datetime.datetime(2012, 10, 1, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 4, 6, 0, 0), 'grade': 'C', 'score': 92}, {'date': datetime.datetime(2011, 11, 3, 0, 0), 'grade': 'C', 'score': 41}], 'name': 'Gandhi', 'restaurant_id': '40381295'}
{'_id': ObjectId('634b6160f53e839bcf876070'), 'address': {'building': '130', 'coord': [-73.984758, 40.7457939], 'street': 'Madison Avenue', 'zipcode': '10016'}, 'borough': 'Manhattan', 'cuisine': 'Pizza/Italian', 'g

13. find the restaurants which does not prepare any cuisine of 'American ' and achieved a grade point 'A' not belongs to the borough Brooklyn. The document must be displayed according to the cuisine in descending order. 

In [ ]:
c1 = db.restaurants.find(
                     {"$query":
                            {
                             "cuisine" : {"$ne" : "American "},
                             "grades.grade" :"A",
                             "borough": "Brooklyn"
                            },
                              "$orderby" : {"cuisine":1}
                      } ,
                      {"name":1,"cuisine":1,"_id":0}
                    );
#for crnt in c1:
#    print(crnt)

14. find the restaurant Id, name, borough and cuisine for those restaurants which contains 'Wil' as first three letters for its name. 


In [ ]:
c1 = db.restaurants.find({"name": {"$regex":"^Wil"}})
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875d12'), 'address': {'building': '7114', 'coord': [-73.9068506, 40.6199034], 'street': 'Avenue U', 'zipcode': '11234'}, 'borough': 'Brooklyn', 'cuisine': 'Delicatessen', 'grades': [{'date': datetime.datetime(2014, 5, 29, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 8, 3, 0, 0), 'grade': 'A', 'score': 8}, {'date': datetime.datetime(2012, 7, 18, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2012, 3, 9, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2011, 10, 14, 0, 0), 'grade': 'A', 'score': 9}], 'name': "Wilken'S Fine Food", 'restaurant_id': '40356483'}
{'_id': ObjectId('634b6160f53e839bcf875d16'), 'address': {'building': '2300', 'coord': [-73.8786113, 40.8502883], 'street': 'Southern Boulevard', 'zipcode': '10460'}, 'borough': 'Bronx', 'cuisine': 'American ', 'grades': [{'date': datetime.datetime(2014, 5, 28, 0, 0), 'grade

In [ ]:
whereCond = {"name": {"$regex":"^Wil"}}
selFields = {
        "restaurant_id" : 2,
        "name":1,"borough":1,
        "cuisine" :1,
         "_id":0
        }
c1 = db.restaurants.find(whereCond,selFields)
for crnt in c1:
    print(crnt)

{'borough': 'Brooklyn', 'cuisine': 'Delicatessen', 'name': "Wilken'S Fine Food", 'restaurant_id': '40356483'}
{'borough': 'Bronx', 'cuisine': 'American ', 'name': 'Wild Asia', 'restaurant_id': '40357217'}
{'borough': 'Bronx', 'cuisine': 'Pizza', 'name': 'Wilbel Pizza', 'restaurant_id': '40871979'}


15.  find the restaurant Id, name, borough and cuisine for those restaurants which contains 'ces' as last three letters for its name.

In [ ]:
c1 = db.restaurants.find(
    {"name": {"$regex":"ces$"}},
        {
        "restaurant_id" : 1,
        "name":1,"borough":1,
        "cuisine" :1
        }
);
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf8761f6'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Pieces', 'restaurant_id': '40399910'}
{'_id': ObjectId('634b6160f53e839bcf8762b6'), 'borough': 'Queens', 'cuisine': 'American ', 'name': 'S.M.R Restaurant Services', 'restaurant_id': '40403857'}
{'_id': ObjectId('634b6160f53e839bcf8762bc'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Good Shepherd Services', 'restaurant_id': '40403989'}
{'_id': ObjectId('634b6160f53e839bcf8767a4'), 'borough': 'Queens', 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices', 'name': "The Ice Box-Ralph'S Famous Italian Ices", 'restaurant_id': '40690899'}
{'_id': ObjectId('634b6160f53e839bcf8769d2'), 'borough': 'Brooklyn', 'cuisine': 'Jewish/Kosher', 'name': 'Alices', 'restaurant_id': '40782042'}
{'_id': ObjectId('634b6160f53e839bcf876bf3'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Re: Sources', 'restaurant_id': '40876068'}


16. find the restaurant Id, name, borough and cuisine for those restaurants which contains 'Reg' as three letters somewhere in its name. 


In [ ]:
c1 = db.restaurants.find(
    {"name": {"$regex":".*Reg.*"}},    
        {
        "restaurant_id" : 1,
        "name":1,"borough":1,
        "cuisine" :1
        }
);
for crnt in c1:
    print(crnt)


{'_id': ObjectId('634b6160f53e839bcf875d14'), 'borough': 'Brooklyn', 'cuisine': 'American ', 'name': 'Regina Caterers', 'restaurant_id': '40356649'}
{'_id': ObjectId('634b6160f53e839bcf875e10'), 'borough': 'Manhattan', 'cuisine': 'Café/Coffee/Tea', 'name': 'Caffe Reggio', 'restaurant_id': '40369418'}
{'_id': ObjectId('634b6160f53e839bcf875f20'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Regency Hotel', 'restaurant_id': '40382679'}
{'_id': ObjectId('634b6160f53e839bcf876294'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Regency Whist Club', 'restaurant_id': '40402377'}
{'_id': ObjectId('634b6160f53e839bcf876378'), 'borough': 'Queens', 'cuisine': 'American ', 'name': 'Rego Park Cafe', 'restaurant_id': '40523342'}
{'_id': ObjectId('634b6160f53e839bcf876a4a'), 'borough': 'Queens', 'cuisine': 'Pizza', 'name': 'Regina Pizza', 'restaurant_id': '40801325'}
{'_id': ObjectId('634b6160f53e839bcf876c63'), 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Regal En

17. find the restaurants which belongs to the borough Bronx and prepared either American or Chinese dish. 


In [ ]:
c1 = db.restaurants.find(
                     {"$query":
                            {
                             "cuisine" :"Chinese",
                             "borough": "Brooklyn"
                            }
                      } ,
                      {
                        "name":1,"borough":1,
                        "cuisine" :1,
                       "_id":0
                      }
                    );
#for crnt in c1:
#    print(crnt)

In [ ]:
c1 = db.restaurants.find(
                     {"$query":
                            {
                             "cuisine" : {"$in" :["Chinese","American "]},
                             "borough": "Brooklyn"
                            }
                      }, 
                      {
                        "name":1,"borough":1,
                        "cuisine" :1,
                       "_id":0
                      }
                    );
#for crnt in c1:
#    print(crnt)

18. find the restaurant Id, name, borough and cuisine for those restaurants which belongs to the borough Staten Island or Queens or Bronxor Brooklyn. 


In [ ]:
c1 = db.restaurants.find(
    {"borough" :{"$in" :["Staten Island","Queens"]}},
        {
        #"restaurant_id" : 1,
        "name":1,"borough":1,
        "cuisine" :1
        }
);
#for crnt in c1:
#    print(crnt)


19. find the restaurant Id, name, borough and cuisine for those restaurants which are not belonging to the borough Staten Island or Queens or Bronxor Brooklyn. 


In [ ]:
c1 = db.restaurants.find(
    {"borough" :{"$nin" :["Staten Island","Queens","Bronx","Brooklyn"]}},
        {
        "restaurant_id" : 1,
        "name":1,"borough":1,
        "cuisine" :1
        }
);
#for crnt in c1:
#    print(crnt)

20. find the restaurant Id, name, borough and cuisine for those restaurants which achieved a score which is not more than 10. 


In [ ]:
c1 = db.restaurants.find(
    {"grades.score" : 
        { "$not": {"$gt" : 10}}
    },
    {
    "restaurant_id" : 1,
    "name":1,"borough":1,
    "cuisine" :1
    }
);
#for crnt in c1:
#    print(crnt)

21. find the restaurant Id, name, borough and cuisine for those restaurants which prepared dish except 'American' and 'Chinese' or restaurant's name begins with letter 'Wil'. 


In [ ]:
c1 = db.restaurants.find(
{"$or": [
  {"name": {"$regex":"^Wil"}}, 
  {"$and": [
       {"cuisine" : {"$ne" :"American "}}, 
       {"cuisine" : {"$ne" :"Chinese"}}
   ]}
]}
,{"restaurant_id" : 1,"name":1,"borough":1,"cuisine" :1}
);

#for crnt in c1:
#    print(crnt)


22. find the restaurant Id, name and grades for those restaurants which achieved a grade of "A" and scored 11 on an ISODate "2014-08-11T00:00:00Z" among many of survey dates. <br>
https://stackoverflow.com/questions/43731016/pymongo-name-isodate-is-not-defined


In [ ]:
import dateutil
dateStr = '2014-08-11T00:00:00Z'
myDatetime = dateutil.parser.parse(dateStr)

In [ ]:
c1 = db.restaurants.find( 
                {
                 "grades.date": myDatetime, 
                 "grades.grade":"A" , 
                 "grades.score" : 11
                }, 
                {"restaurant_id" : 1,"name":1,"grades":1}
             );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875d8a'), 'grades': [{'date': datetime.datetime(2014, 8, 11, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2013, 7, 22, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 3, 14, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2012, 7, 2, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2012, 2, 2, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2011, 8, 24, 0, 0), 'grade': 'A', 'score': 11}], 'name': "Neary'S Pub", 'restaurant_id': '40365871'}
{'_id': ObjectId('634b6160f53e839bcf875e64'), 'grades': [{'date': datetime.datetime(2014, 8, 11, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2013, 12, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 6, 10, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2012, 6, 8, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2012, 1, 25, 0, 0), 'grade': 'A', 'score': 8}, {'date': date

23. find the restaurant Id, name and grades for those restaurants where 2nd element of grades array contains a grade of "A" and score 9 on an ISODate "2014-08-11T00:00:00Z". 


In [ ]:
import dateutil
dateStr = '2014-08-11T00:00:00Z'
myDatetime = dateutil.parser.parse(dateStr)

c1 = db.restaurants.find( 
                      { "grades.1.date": myDatetime, 
                        "grades.1.grade":"A" , 
                        "grades.1.score" : 9
                      }, 
                       {"restaurant_id" : 1,"name":1,"grades":1}
                   );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf876390'), 'grades': [{'date': datetime.datetime(2015, 1, 12, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2014, 8, 11, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2014, 1, 14, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2013, 2, 7, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2012, 4, 30, 0, 0), 'grade': 'A', 'score': 11}], 'name': 'Club Macanudo (Cigar Bar)', 'restaurant_id': '40526406'}


24. find the restaurant Id, name, address and geographical location for those restaurants where 2nd element of coord array contains a value which is more than 42 and upto 52. 


In [ ]:
c1 = db.restaurants.find( 
                      { 
                        "address.coord.1": {"$gt" : 42, "$lte" : 52}
                      },
                        {"restaurant_id" : 1,"name":1,"address":1,"coord":1}
                   );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875faf'), 'address': {'building': '47', 'coord': [-78.877224, 42.89546199999999], 'street': 'Broadway @ Trinity Pl', 'zipcode': '10006'}, 'name': "T.G.I. Friday'S", 'restaurant_id': '40387990'}
{'_id': ObjectId('634b6160f53e839bcf875fdc'), 'address': {'building': '1', 'coord': [-0.7119979, 51.6514664], 'street': 'Pennplaza E, Penn Sta', 'zipcode': '10001'}, 'name': 'T.G.I. Fridays', 'restaurant_id': '40388936'}
{'_id': ObjectId('634b6160f53e839bcf87628a'), 'address': {'building': '3000', 'coord': [-87.86567699999999, 42.61150920000001], 'street': '47 Avenue', 'zipcode': '11101'}, 'name': "Di Luvio'S Deli", 'restaurant_id': '40402284'}
{'_id': ObjectId('634b6160f53e839bcf8764c1'), 'address': {'building': '21972199', 'coord': [-78.589606, 42.8912372], 'street': 'Broadway', 'zipcode': '10024'}, 'name': 'La Caridad 78', 'restaurant_id': '40568285'}
{'_id': ObjectId('634b6160f53e839bcf876bf7'), 'address': {'building': '7981', 'coord': [-84.9751215, 45.471

25. arrange the name of the restaurants in ascending order along with all the columns. <hr>

https://stackoverflow.com/questions/57778658/typeerror-if-no-direction-is-specified-key-or-list-must-be-an-instance-of-list


In [ ]:
c1 = db.restaurants.find({},{"restaurant_id" : 2,"name":1,"_id":0}).sort("name", -1).limit(20);
for crnt in c1:
    print(crnt)

{'name': 'Zum Stammtisch', 'restaurant_id': '40367377'}
{'name': 'Zum Schneider', 'restaurant_id': '40763382'}
{'name': "Zorba'S", 'restaurant_id': '40877247'}
{'name': 'Zebu Grill', 'restaurant_id': '40800325'}
{'name': "Zaro'S Bread Basket", 'restaurant_id': '40397863'}
{'name': "Zaro'S Bread Basket", 'restaurant_id': '40510389'}
{'name': "Zaro'S Bread Basket", 'restaurant_id': '40512617'}
{'name': "Zaro'S Bread Basket", 'restaurant_id': '40586000'}
{'name': "Zaro'S Bread Basket", 'restaurant_id': '40750766'}
{'name': "Zaro'S Bakery", 'restaurant_id': '40423705'}
{'name': "Zaro'S Bakery", 'restaurant_id': '40686674'}
{'name': "Zafi'S Luncheonette", 'restaurant_id': '40371718'}
{'name': 'Yvonne Yvonne Restaurant', 'restaurant_id': '40875849'}
{'name': 'Yura & Company On Madison', 'restaurant_id': '40646987'}
{'name': 'Yummy Kitchen', 'restaurant_id': '40763160'}
{'name': 'Your Bakery', 'restaurant_id': '40529415'}
{'name': 'Yonah Shimmels Knishes', 'restaurant_id': '40544576'}
{'name'

26.  arrange the name of the restaurants in descending along with all the columns. 


In [ ]:
c1 = db.restaurants.find().sort("name", -1).limit(10);
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875dcb'), 'address': {'building': '6946', 'coord': [-73.8811834, 40.7017759], 'street': 'Myrtle Avenue', 'zipcode': '11385'}, 'borough': 'Queens', 'cuisine': 'German', 'grades': [{'date': datetime.datetime(2014, 9, 24, 0, 0), 'grade': 'A', 'score': 11}, {'date': datetime.datetime(2014, 4, 17, 0, 0), 'grade': 'A', 'score': 7}, {'date': datetime.datetime(2013, 3, 12, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2012, 10, 2, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2012, 5, 9, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2011, 12, 28, 0, 0), 'grade': 'B', 'score': 24}], 'name': 'Zum Stammtisch', 'restaurant_id': '40367377'}
{'_id': ObjectId('634b6160f53e839bcf8769be'), 'address': {'building': '107109', 'coord': [-73.9744668, 40.731155], 'street': 'Avenue C', 'zipcode': '10009'}, 'borough': 'Manhattan', 'cuisine': 'German', 'grades': [{'date': datetime.datetime(2014, 3, 4, 0, 0), 'grade': 'A', 'score'

28. Write a MongoDB query to know whether all the addresses contains the street or not. 


In [ ]:
c1 = db.restaurants.find(
                     {"address.street" : 
                         { "$exists" : "true" } 
                     } 
                   );
#for crnt in c1:
#    print(crnt)

29. Write a MongoDB query which will select all documents in the restaurants collection where the coord field value is Double. 


In [ ]:
c1 = db.restaurants.find(
                    {"address.coord" : 
                       {"$type" : 1}
                    }
                   );
#for crnt in c1:
#    print(crnt)

30. select the restaurant Id, name and grades for those restaurants which returns 0 as a remainder after dividing the score by 7. 

In [ ]:
c1 = db.restaurants.find(
                      {"grades.score" :
                         {"$mod" : [7,0]}
                      },
                         {"restaurant_id" : 1,"name":1,"grades":1}
                    );
#for crnt in c1:
#    print(crnt)

31. find the restaurant name, borough, longitude and attitude and cuisine for those restaurants which contains 'mon' as three letters somewhere in its name. 

In [ ]:
c1 = db.restaurants.find(
                   { "name" : 
                     { "$regex" : "mon.*", "$options": "i" } 
                   },
                       {
                         "name":1,
                         "borough":1,
                         "address.coord":1,
                         "cuisine" :1
                        }
                   );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf875d9f'), 'address': {'coord': [-73.98306099999999, 40.7441419]}, 'borough': 'Manhattan', 'cuisine': 'American ', 'name': "Desmond'S Tavern"}
{'_id': ObjectId('634b6160f53e839bcf875da9'), 'address': {'coord': [-73.8221418, 40.7272376]}, 'borough': 'Queens', 'cuisine': 'Jewish/Kosher', 'name': 'Shimons Kosher Pizza'}
{'_id': ObjectId('634b6160f53e839bcf875db5'), 'address': {'coord': [-74.10465599999999, 40.58834]}, 'borough': 'Staten Island', 'cuisine': 'American ', 'name': 'Richmond County Country Club'}
{'_id': ObjectId('634b6160f53e839bcf875ddf'), 'address': {'coord': [-73.9812843, 40.5947365]}, 'borough': 'Brooklyn', 'cuisine': 'Pizza/Italian', 'name': 'Lb Spumoni Gardens'}
{'_id': ObjectId('634b6160f53e839bcf875e32'), 'address': {'coord': [-73.951199, 40.7166026]}, 'borough': 'Brooklyn', 'cuisine': 'Italian', 'name': "Bamonte'S Restaurant"}
{'_id': ObjectId('634b6160f53e839bcf875e6a'), 'address': {'coord': [-73.924072, 40.76108900000001]}, 'borou

32. find the restaurant name, borough, longitude and latitude and cuisine for those restaurants which contains 'Mad' as first three letters of its name. 

In [ ]:
c1 = db.restaurants.find(
                   { "name" : 
                     { "$regex" : "^Mad", } 
                   },
                       {
                         "name":1,
                         "borough":1,
                         "address.coord":1,
                         "cuisine" :1
                        }
                   );
for crnt in c1:
    print(crnt)

{'_id': ObjectId('634b6160f53e839bcf8762a0'), 'address': {'coord': [-73.9860597, 40.7431194]}, 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Madison Square'}
{'_id': ObjectId('634b6160f53e839bcf87636e'), 'address': {'coord': [-73.98302199999999, 40.742313]}, 'borough': 'Manhattan', 'cuisine': 'Indian', 'name': 'Madras Mahal'}
{'_id': ObjectId('634b6160f53e839bcf87664f'), 'address': {'coord': [-74.000002, 40.72735]}, 'borough': 'Manhattan', 'cuisine': 'American ', 'name': 'Madame X'}
{'_id': ObjectId('634b6160f53e839bcf8766ff'), 'address': {'coord': [-73.98171959999999, 40.7499406]}, 'borough': 'Manhattan', 'cuisine': 'French', 'name': 'Madison Bistro'}
{'_id': ObjectId('634b6160f53e839bcf87678a'), 'address': {'coord': [-73.9717845, 40.6897199]}, 'borough': 'Brooklyn', 'cuisine': 'African', 'name': 'Madiba'}
{'_id': ObjectId('634b6160f53e839bcf876abd'), 'address': {'coord': [-73.9040753, 40.9069011]}, 'borough': 'Bronx', 'cuisine': 'Italian', 'name': "Madison'S"}
{'_id': Obje

# Checked by

In [ ]:
from datetime import datetime
import pytz
print('last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

last tested   2022-12-12 10:12:06.921768+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)